In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine, CspaceFreeRegion
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.4
x_fac = 1.2
locs = [ [0,0,0], 
        [x_fac*sp, 1.4*sp,0.4], [x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,1.4*sp,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros_like(plant.GetPositionLowerLimits())

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram


DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/shelves.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_prm_certify.runfiles/drake/C_Iris_Examples/assets/shelves.sdf' (good)
DEB

In [6]:
visualizer.meshcat2.Delete("/prm")

In [7]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 20,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = 10,
            num_col_checks = 10,
            max_it = 1e5,
            verbose = True,
            plotcallback = None
            )

# PRM_bad = prm.PRM( 
#             limits_t,
#             num_points = 2000,
#             col_func_handle = prm_col_fn_handle_bad,
#             num_neighbours = 5, 
#             dist_thresh = .5,
#             num_col_checks = 10,
#             verbose = True,
#             plotcallback = None
#             )

# PRM.add_start_end(start, target)
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])

[PRM] Samples 0
[PRM] Nodes connected: 0


In [8]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
# s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
# print(s0_mat_unsafe.shape)

(14, 7)


In [9]:
PRM.adjacency_list

[[0],
 [1],
 [2, 13],
 [3, 15, 16],
 [4],
 [5, 7],
 [6],
 [7, 5],
 [8],
 [9, 11],
 [10],
 [11, 9, 18],
 [12],
 [13, 2],
 [14],
 [15, 18, 3],
 [16, 3],
 [17],
 [18, 15, 11],
 [19]]

## Set up the sliders so we can move the plant around manually

In [10]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q6', max=3.05433, min=-3.05433)

In [11]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(f"Number of pairs {len(pairs)}")


Number of pairs 42


In [12]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe
def make_certifier(use_affine = True, num_points = -1):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    q_star,
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    return line_certifier

def do_certify(line_certifier, num_points = -1):
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe[:num_points,:], s1_mat_safe[:num_points,:])
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
    return (safe_points, planes, num_safe1)

In [13]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, 
                                q_star,
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

DEBUG:drake:Done rational 1/672
DEBUG:drake:Done rational 2/672
DEBUG:drake:Done rational 3/672
DEBUG:drake:Done rational 4/672
DEBUG:drake:Done rational 5/672
DEBUG:drake:Done rational 6/672
DEBUG:drake:Done rational 7/672
DEBUG:drake:Done rational 8/672
DEBUG:drake:Done rational 9/672
DEBUG:drake:Done rational 10/672
DEBUG:drake:Done rational 11/672
DEBUG:drake:Done rational 12/672
DEBUG:drake:Done rational 13/672
DEBUG:drake:Done rational 14/672
DEBUG:drake:Done rational 15/672
DEBUG:drake:Done rational 16/672
DEBUG:drake:Done rational 17/672
DEBUG:drake:Done rational 18/672
DEBUG:drake:Done rational 19/672
DEBUG:drake:Done rational 20/672
DEBUG:drake:Done rational 21/672
DEBUG:drake:Done rational 22/672
DEBUG:drake:Done rational 23/672
DEBUG:drake:Done rational 24/672
DEBUG:drake:Done rational 25/672
DEBUG:drake:Done rational 26/672
DEBUG:drake:Done rational 27/672
DEBUG:drake:Done rational 28/672
DEBUG:drake:Done rational 29/672
DEBUG:drake:Done rational 30/672
DEBUG:drake:Done ra

DEBUG:drake:Done rational 246/672
DEBUG:drake:Done rational 247/672
DEBUG:drake:Done rational 248/672
DEBUG:drake:Done rational 249/672
DEBUG:drake:Done rational 250/672
DEBUG:drake:Done rational 251/672
DEBUG:drake:Done rational 252/672
DEBUG:drake:Done rational 253/672
DEBUG:drake:Done rational 254/672
DEBUG:drake:Done rational 255/672
DEBUG:drake:Done rational 256/672
DEBUG:drake:Done rational 257/672
DEBUG:drake:Done rational 258/672
DEBUG:drake:Done rational 259/672
DEBUG:drake:Done rational 260/672
DEBUG:drake:Done rational 261/672
DEBUG:drake:Done rational 262/672
DEBUG:drake:Done rational 263/672
DEBUG:drake:Done rational 264/672
DEBUG:drake:Done rational 265/672
DEBUG:drake:Done rational 266/672
DEBUG:drake:Done rational 267/672
DEBUG:drake:Done rational 268/672
DEBUG:drake:Done rational 269/672
DEBUG:drake:Done rational 270/672
DEBUG:drake:Done rational 271/672
DEBUG:drake:Done rational 272/672
DEBUG:drake:Done rational 273/672
DEBUG:drake:Done rational 274/672
DEBUG:drake:Do

DEBUG:drake:Done rational 487/672
DEBUG:drake:Done rational 488/672
DEBUG:drake:Done rational 489/672
DEBUG:drake:Done rational 490/672
DEBUG:drake:Done rational 491/672
DEBUG:drake:Done rational 492/672
DEBUG:drake:Done rational 493/672
DEBUG:drake:Done rational 494/672
DEBUG:drake:Done rational 495/672
DEBUG:drake:Done rational 496/672
DEBUG:drake:Done rational 497/672
DEBUG:drake:Done rational 498/672
DEBUG:drake:Done rational 499/672
DEBUG:drake:Done rational 500/672
DEBUG:drake:Done rational 501/672
DEBUG:drake:Done rational 502/672
DEBUG:drake:Done rational 503/672
DEBUG:drake:Done rational 504/672
DEBUG:drake:Done rational 505/672
DEBUG:drake:Done rational 506/672
DEBUG:drake:Done rational 507/672
DEBUG:drake:Done rational 508/672
DEBUG:drake:Done rational 509/672
DEBUG:drake:Done rational 510/672
DEBUG:drake:Done rational 511/672
DEBUG:drake:Done rational 512/672
DEBUG:drake:Done rational 513/672
DEBUG:drake:Done rational 514/672
DEBUG:drake:Done rational 515/672
DEBUG:drake:Do

DEBUG:drake:constructing tuple 50/672
DEBUG:drake:constructing tuple 51/672
DEBUG:drake:constructing tuple 52/672
DEBUG:drake:constructing tuple 53/672
DEBUG:drake:constructing tuple 54/672
DEBUG:drake:constructing tuple 55/672
DEBUG:drake:constructing tuple 56/672
DEBUG:drake:constructing tuple 57/672
DEBUG:drake:constructing tuple 58/672
DEBUG:drake:constructing tuple 59/672
DEBUG:drake:constructing tuple 60/672
DEBUG:drake:constructing tuple 61/672
DEBUG:drake:constructing tuple 62/672
DEBUG:drake:constructing tuple 63/672
DEBUG:drake:constructing tuple 64/672
DEBUG:drake:constructing tuple 65/672
DEBUG:drake:constructing tuple 66/672
DEBUG:drake:constructing tuple 67/672
DEBUG:drake:constructing tuple 68/672
DEBUG:drake:constructing tuple 69/672
DEBUG:drake:constructing tuple 70/672
DEBUG:drake:constructing tuple 71/672
DEBUG:drake:constructing tuple 72/672
DEBUG:drake:constructing tuple 73/672
DEBUG:drake:constructing tuple 74/672
DEBUG:drake:constructing tuple 75/672
DEBUG:drake:

DEBUG:drake:constructing tuple 262/672
DEBUG:drake:constructing tuple 263/672
DEBUG:drake:constructing tuple 264/672
DEBUG:drake:constructing tuple 265/672
DEBUG:drake:constructing tuple 266/672
DEBUG:drake:constructing tuple 267/672
DEBUG:drake:constructing tuple 268/672
DEBUG:drake:constructing tuple 269/672
DEBUG:drake:constructing tuple 270/672
DEBUG:drake:constructing tuple 271/672
DEBUG:drake:constructing tuple 272/672
DEBUG:drake:constructing tuple 273/672
DEBUG:drake:constructing tuple 274/672
DEBUG:drake:constructing tuple 275/672
DEBUG:drake:constructing tuple 276/672
DEBUG:drake:constructing tuple 277/672
DEBUG:drake:constructing tuple 278/672
DEBUG:drake:constructing tuple 279/672
DEBUG:drake:constructing tuple 280/672
DEBUG:drake:constructing tuple 281/672
DEBUG:drake:constructing tuple 282/672
DEBUG:drake:constructing tuple 283/672
DEBUG:drake:constructing tuple 284/672
DEBUG:drake:constructing tuple 285/672
DEBUG:drake:constructing tuple 286/672
DEBUG:drake:constructing 

DEBUG:drake:constructing tuple 473/672
DEBUG:drake:constructing tuple 474/672
DEBUG:drake:constructing tuple 475/672
DEBUG:drake:constructing tuple 476/672
DEBUG:drake:constructing tuple 477/672
DEBUG:drake:constructing tuple 478/672
DEBUG:drake:constructing tuple 479/672
DEBUG:drake:constructing tuple 480/672
DEBUG:drake:constructing tuple 481/672
DEBUG:drake:constructing tuple 482/672
DEBUG:drake:constructing tuple 483/672
DEBUG:drake:constructing tuple 484/672
DEBUG:drake:constructing tuple 485/672
DEBUG:drake:constructing tuple 486/672
DEBUG:drake:constructing tuple 487/672
DEBUG:drake:constructing tuple 488/672
DEBUG:drake:constructing tuple 489/672
DEBUG:drake:constructing tuple 490/672
DEBUG:drake:constructing tuple 491/672
DEBUG:drake:constructing tuple 492/672
DEBUG:drake:constructing tuple 493/672
DEBUG:drake:constructing tuple 494/672
DEBUG:drake:constructing tuple 495/672
DEBUG:drake:constructing tuple 496/672
DEBUG:drake:constructing tuple 497/672
DEBUG:drake:constructing 

Time to construct line certifier = 328.6434762477875s


In [14]:
i=0
t0 = time.time()
safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLine(s0_mat_safe[i,:], s1_mat_safe[i,:])
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

DEBUG:drake:doing constraint 1/42
DEBUG:drake:doing constraint 2/42
DEBUG:drake:doing constraint 3/42
DEBUG:drake:doing constraint 4/42
DEBUG:drake:doing constraint 5/42
DEBUG:drake:doing constraint 6/42
DEBUG:drake:doing constraint 7/42
DEBUG:drake:doing constraint 8/42
DEBUG:drake:doing constraint 9/42
DEBUG:drake:doing constraint 10/42
DEBUG:drake:doing constraint 11/42
DEBUG:drake:doing constraint 12/42
DEBUG:drake:doing constraint 13/42
DEBUG:drake:doing constraint 14/42
DEBUG:drake:doing constraint 15/42
DEBUG:drake:doing constraint 16/42
DEBUG:drake:doing constraint 17/42
DEBUG:drake:doing constraint 18/42
DEBUG:drake:doing constraint 19/42
DEBUG:drake:doing constraint 20/42
DEBUG:drake:doing constraint 21/42
DEBUG:drake:doing constraint 22/42
DEBUG:drake:doing constraint 23/42
DEBUG:drake:doing constraint 24/42
DEBUG:drake:doing constraint 25/42
DEBUG:drake:doing constraint 26/42
DEBUG:drake:doing constraint 27/42
DEBUG:drake:doing constraint 28/42
DEBUG:drake:doing constraint 

Certification of safe PRM in 55.96195387840271s
